In [1]:
import json
import pandas as pd
from datasets import Dataset
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import numpy as np


In [ ]:
# 파인튜닝 데이터셋 읽어오기

FILE_PATH = "../../data/fine-tuning-data/KLUE-sts"

def extract_essential_fields(data):
    rows = []
    for item in data:
        rows.append({
            "sentence1": item["sentence1"],
            "sentence2": item["sentence2"],
            "labels": float(item["labels"]["label"])
        })
    return pd.DataFrame(rows)

with open(f"{FILE_PATH}/klue-sts-v1.1_train.json", encoding='utf-8') as f:
    train_data = json.load(f)

with open(f"{FILE_PATH}/klue-sts-v1.1_dev.json", encoding='utf-8') as f:
    val_data = json.load(f)

train_df = extract_essential_fields(train_data)
val_df = extract_essential_fields(val_data)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "klue/roberta-base",
    num_labels=1 # 1이면 float값 출력, 0이면 이진 분류
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def preprocess(example):
    return tokenizer(
        example["sentence1"],
        example["sentence2"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

train_dataset = train_dataset.map(preprocess, batched=True)
val_dataset = val_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/11668 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

In [5]:
# Trainer가 학습에 쓸 수 있도록 label 설정
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [6]:
def compute_metrics(pred):
    preds, labels = pred
    preds = preds.flatten()
    labels = labels.flatten()
    
    pearson = pearsonr(preds, labels)[0]
    mse = mean_squared_error(labels, preds)
    mae = mean_absolute_error(labels, preds)

    return {
        'pearson': pearson,
        'mse': mse,
        'mae': mae
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="C:/results", # 경로에 한글이 있으면 안돼서 이렇게 구성하였음. 수정 가능
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="pearson",
    save_total_limit=1
)


c:\Users\to765\anaconda3\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\to765\AppData\Local\Temp\ipykernel_30620\842634874.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
# 1. 학습 전 평가
print("📊 학습 전 평가 결과:")
trainer.evaluate()

# 2. 학습
trainer.train()

# 3. 학습 후 평가
print("📊 학습 후 평가 결과:")
trainer.evaluate()


📊 학습 전 평가 결과:


Epoch,Training Loss,Validation Loss,Model Preparation Time,Pearson,Mse,Mae
1,0.524000,0.519043,0.000000,0.903155,0.519043,0.529355
2,0.176700,0.379416,0.000000,0.922982,0.379416,0.451566
3,0.107400,0.487042,0.000000,0.914861,0.487042,0.519524
4,0.085900,0.310401,0.000000,0.926843,0.310401,0.421881
5,0.056700,0.402473,0.000000,0.927653,0.402473,0.465382
6,0.049800,0.322850,0.000000,0.932446,0.322850,0.419430
7,0.037000,0.368098,0.000000,0.925525,0.368098,0.452963
8,0.031900,0.320194,0.000000,0.932796,0.320194,0.415787
9,0.025000,0.371158,0.000000,0.929914,0.371158,0.453890
10,0.022500,0.337652,0.000000,0.932425,0.337652,0.431542


📊 학습 후 평가 결과:


{'eval_loss': 0.32019418478012085,
 'eval_model_preparation_time': 0.0,
 'eval_pearson': 0.9327959280897391,
 'eval_mse': 0.3201942443847656,
 'eval_mae': 0.41578707098960876,
 'eval_runtime': 1.2293,
 'eval_samples_per_second': 422.179,
 'eval_steps_per_second': 26.844,
 'epoch': 10.0}

In [ ]:
predictions = trainer.predict(val_dataset)
preds = predictions.predictions.squeeze()
labels = predictions.label_ids.squeeze()

# 예시: 판다스로 저장
import pandas as pd

df_result = pd.DataFrame({
    "sentence1": val_df["sentence1"],
    "sentence2": val_df["sentence2"],
    "gold_label": labels,
    "predicted": preds
})

df_result.to_csv("./results/klue-sts-predictions.csv", index=False)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_path = "C:/results/checkpoint-5840" # 개인에 맞게 수정 필요

# 모델과 토크나이저 로딩
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()  # 평가 모드

# 예측 함수
def predict_similarity(sentence1, sentence2):
    inputs = tokenizer(
        sentence1,
        sentence2,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )

    with torch.no_grad():
        outputs = model(**inputs)
        score = outputs.logits.squeeze().item()

    return max(0.0, min(5.0, score))  # 점수 0~5로 클리핑


s1 = "르네상스 시대에 자신들과 중세를 구별하면서 시작되었으며, 카를 마르크스의 역사 발전 단계 이론이 나온 후, 경제적 발달 수준에 따라서 구별하는 추세가 우세하다."
s2 = "일반적인 정의는 '나무에서 나는 단맛이 포함된 식용 가능한 열매'이다. 풀에서 수확하는 열매는 야채, 나무에서 수확하는 열매는 과일로 분류하기도 한다. "

similarity = predict_similarity(s1, s2)
print(f"📈 예측된 유사도: {similarity:.2f} / 5.0")


📈 예측된 유사도: 0.27 / 5.0
